In [1]:
# imports
import sys
import os
import socket
import pandas as pd
import glob
from datetime import datetime

In [2]:
# function to format dates
def formatdate(data):
    data['sysdate'] = data['sysdate'].str.replace('/', '')
    data['sysdate']= pd.to_datetime(data['sysdate'], format='%d%m%Y')
    data['eventdate']= data['eventdate'].str.replace('/', '')
    data['eventdate'] = pd.to_datetime(data['eventdate'], format='%d%m%Y')
    data['eventdate'].fillna(data['sysdate'], inplace = True)
    return data

In [3]:
os.getcwd()

'/mnt/maths/tws21'

In [4]:
next(os.walk('/mnt/maths/tws21/Dementia Symptoms Codes'))

('/mnt/maths/tws21/Dementia Symptoms Codes',
 [],
 ['violent or agressive behaviour.csv',
  'repetition.xlsx',
  'referral psych neuro or geri.csv',
  'Referral for dementia diagnosis.xlsx',
  'personality change.csv',
  'MMSE.csv',
  'memory loss codes.csv',
  'family history of dementia.csv',
  'does not take medication.csv',
  'disorientation wandering.csv',
  'depressive symptoms.csv',
  'cognitive decline codelist.csv',
  'behaviour change.csv',
  'cognitive screening tests.csv'])

In [5]:
# read in symptom files for dementia
behaviour_change = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/behaviour change.csv', sep = '\t')
cognitive_tests = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/cognitive screening tests.csv', sep = '\t')
cognitive_decline = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/cognitive decline codelist.csv',
                                sep = '\t')
depressive_symptoms = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/depressive symptoms.csv', sep = '\t')
disorientation = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/disorientation wandering.csv', sep = '\t')
family_history = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/family history of dementia.csv', sep = '\t')
memory_loss = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/memory loss codes.csv', sep = '\t')
mmse = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/MMSE.csv', sep = '\t')
personality = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/personality change.csv', sep = '\t')
referral_psych = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/referral psych neuro or geri.csv', sep = '\t')
skip_meds = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/does not take medication.csv', sep = '\t')
violent = pd.read_csv('/mnt/maths/tws21/Dementia Symptoms Codes/violent or agressive behaviour.csv', sep = '\t')

In [6]:
# Merge all symptom files (note: only include the .csv files as the others are read code only i.e. HES)
symptoms_merged = pd.concat([behaviour_change, cognitive_tests, cognitive_decline, depressive_symptoms,
                             disorientation, family_history, memory_loss, mmse, personality, referral_psych,
                             skip_meds, violent])
symptoms_merged.head()

,medcode,clinicalevents,referralevents,testevents,immunisationevents,readcode,readterm,databasebuild
0,3566,26676,3948,0,0,1469.00,H/O: behaviour problem,February 2009
1,11054,42690,3339,0,0,1B1X.00,Behavioural problem,February 2009
2,18893,1247,86,0,0,1P...00,Behaviours and observations relating to behaviour,February 2009
3,47903,79,6,0,0,1P0..00,Motor function behaviour,February 2009
4,10918,6340,691,0,0,1P00.00,Hyperactive behaviour,February 2009


In [7]:
# read in dementia codes
dementia_codes = pd.read_excel('/mnt/maths/tws21/dementia_codes.xlsx')

In [8]:
# select non diagnosis codes (admin, tests etc)
dementia_med_codes = dementia_codes[dementia_codes['diagnosis'] != 1.0]['medcode']

In [9]:
dementia_med_codes.head()

34      9385
35    107650
62     10288
66     26141
67     26421
Name: medcode, dtype: int64

In [10]:
symptom_med_codes = symptoms_merged['medcode']

In [11]:
#read in and merge the referral files and search for med codes
path = '/mnt/lustre/users/maths/tws21/seb_tmp/bsms2730/bsms2730/Primary Care/CPRD GOLD/'

referral_files = glob.glob(path + "*referral*.txt")

referral_li = []
for filename in referral_files:
    df = pd.read_csv(filename, sep="\t")
    df = df[df['medcode'].isin(list(dementia_med_codes) + list(symptom_med_codes))]
    df = formatdate(df)
    referral_li.append(df)

referral_merged = pd.concat(referral_li, axis=0)
referral_merged.head()

,e_patid,eventdate,sysdate,constype,consid,medcode,nhsspec,attendance,urgency
11,10009416,2003-04-04,2009-02-20,4,1067550,2964,0,0,0
38,10010191,2000-01-18,2000-01-20,4,136437,5777,54,1,4
308,100305191,2004-07-01,2004-07-01,4,476673,10448,36,0,2
379,10036782,2011-03-16,2011-03-16,4,168769,1511,0,0,2
386,10036782,2013-03-04,2013-03-04,4,169048,2189,54,0,4


In [13]:
# save to parquet
referral_merged.to_parquet(os.getcwd() + '/dementia_symptoms_referral_data.parquet')

In [15]:
referral_merged.shape[0]

122785